In [1]:
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ExponentialLR
from torchvision import models, transforms, datasets
from torchvision.io import read_image
from torchsummary import summary
import numpy as np
import pandas as pd
from collections import defaultdict
from shutil import copy

Constants

In [2]:
FOOD_PATH = "./dataset/food-101"
IMG_PATH = FOOD_PATH+"/images"
META_PATH = FOOD_PATH+"/meta"
TRAIN_PATH = FOOD_PATH+"/train"
VALID_PATH = FOOD_PATH+"/valid"
MODEL_PATH = 'model_data/'
IMAGENET_STATS = [(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)]
BATCH_SIZE = 500
EPOCHS = 5

Making dataloaders for the dataset

In [3]:
class FOOD101():
    def __init__(self):
        self.train_ds, self.valid_ds, self.train_cls, self.valid_cls = [None]*4
        self.imgenet_mean = IMAGENET_STATS[0]
        self.imgenet_std = IMAGENET_STATS[1]
        
    def _get_tfms(self):
        train_tfms = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(), 
            transforms.ToTensor(),
            transforms.Normalize(self.imgenet_mean, self.imgenet_std)])
        
        valid_tfms = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(self.imgenet_mean, self.imgenet_std)])        
        return train_tfms, valid_tfms            
            
    def get_dataset(self,root_dir='dataset/food-101/'):
        train_tfms, valid_tfms = self._get_tfms() # transformations
        self.train_ds = datasets.ImageFolder(root=TRAIN_PATH, transform=train_tfms)
        self.valid_ds = datasets.ImageFolder(root=VALID_PATH, transform=valid_tfms)        
        self.train_classes = self.train_ds.classes
        self.valid_classes = self.valid_ds.classes

        assert self.train_classes==self.valid_classes
        return self.train_ds, self.valid_ds, self.train_classes

    
    def get_dls(self, train_ds, valid_ds, bs, **kwargs):
        return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
               DataLoader(valid_ds, batch_size=bs//2, shuffle=False, **kwargs))

In [4]:
food = FOOD101() 
train_ds, valid_ds, classes = food.get_dataset()
dls = food.get_dls(train_ds, valid_ds, BATCH_SIZE)
train_dl, valid_dl = dls[0], dls[1]

MODEL

In [5]:
def build_model():
    layers = []

    # in_channels, out_channels, kernel_size
    layers.append(nn.Conv2d(3, 5, 3))
    layers.append(nn.MaxPool2d(3))

    layers.append(nn.Conv2d(5, 5, 3))
    layers.append(nn.MaxPool2d(3))

    layers.append(nn.Conv2d(5, 5, 3))
    layers.append(nn.MaxPool2d(3, stride=1))
    
    layers.append(nn.Conv2d(5, 5, 3))
    layers.append(nn.MaxPool2d(3, stride=1))
    
    layers.append(nn.Conv2d(5, 5, 3))
    layers.append(nn.MaxPool2d(3, stride=1))
    
    layers.append(nn.Conv2d(5, 5, 3))
    layers.append(nn.MaxPool2d(3))
    
    layers.append(nn.Flatten())

    layers.append(nn.Linear(45, 80))
    layers.append(nn.ReLU())

    layers.append(nn.Linear(80, 101))
    layers.append(nn.ReLU())
    layers.append(nn.Softmax())
    
    model = nn.Sequential(*layers)
    
    return model

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
def train(model, train_dataloader, test_dataloader, learning_rate = 0.9):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    scheduler = ExponentialLR(optimizer, gamma=0.9)

    for t in range(EPOCHS):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        test_loop(test_dataloader, model, loss_fn)
        scheduler.step()
    print("Done!")

In [9]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
device

device(type='cuda')

In [10]:
model = build_model().to(device)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 5, 217, 217]             965
         MaxPool2d-2          [-1, 5, 210, 210]               0
            Conv2d-3          [-1, 5, 203, 203]           1,605
         MaxPool2d-4          [-1, 5, 196, 196]               0
            Conv2d-5          [-1, 5, 189, 189]           1,605
         MaxPool2d-6          [-1, 5, 182, 182]               0
            Conv2d-7          [-1, 5, 175, 175]           1,605
         MaxPool2d-8          [-1, 5, 168, 168]               0
            Conv2d-9          [-1, 5, 161, 161]           1,605
        MaxPool2d-10          [-1, 5, 154, 154]               0
           Conv2d-11          [-1, 5, 147, 147]           1,605
        MaxPool2d-12            [-1, 5, 18, 18]               0
          Flatten-13                 [-1, 1620]               0
           Linear-14                  [

/home/raha/anaconda3/lib/python3.8/site-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
train(model, train_dl, valid_dl)

Epoch 1
-------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 632.00 MiB (GPU 0; 5.79 GiB total capacity; 4.06 GiB already allocated; 238.00 MiB free; 4.25 GiB reserved in total by PyTorch)